In [1]:
""" 1 Load Packages & def checkXML """
import xml.dom.minidom
import numpy as np
import os
from skimage.measure import label, regionprops
from PIL import Image, ImageDraw

def checkXML(filelist):
    status = None
    for f in filelist:
        if f.split('.')[-1] == 'xml':
            status = f
    return status

In [2]:
""" 2 Function CutAndLabel """
class_list = {'Cloud':1, 'Crystal':2, 'Feather':3, 'Needle':4, 'Twinning_wisp':5}

def CutAndLabel(xmlpath,src_path,save_path,txt_save_path,resize=False,tar_size=(32,32),clear_dist=10):
    dom = xml.dom.minidom.parse(os.path.join(src_path,xmlpath))
    xml_root = dom.documentElement
    images = xml_root.getElementsByTagName('image')

    # # Note: pinpoint is not considered here # 'Pinpoint':1,
    # class_list = {'Pinpoint': 1, 'Crystal': 2, 'Needle': 3, 'Feather': 4, 'Internal_graining': 5,
    #                        'Cloud': 6, 'Twinning_wisp': 7, 'Nick': 8, 'Pit': 9, 'Burn_mark': 10}
    # above Before 2022-10-03
    class_list = {'Cloud':1, 'Crystal':2, 'Feather':3, 'Needle':4, 'Twinning_wisp':5}

    for ii in range(len(images)):
        t = images[ii]
        t_name = t.getAttribute("name").split('/')[-1]
        t_width = int(t.getAttribute("width"))
        t_height = int(t.getAttribute("height"))

        label_file = open(os.path.join(txt_save_path,t_name[:11]+'.txt'),'w')

        try:
            img_name = [n for n in os.listdir(src_path) if t_name[:11] in n and 'png' in n][0]
            img = Image.open(os.path.join(src_path,img_name)).convert('RGB')
        except:
            img_name = [n for n in os.listdir(os.path.join(src_path,'Twinning wisp')) if t_name[:11] in n and 'png' in n][0]
            img = Image.open(os.path.join(src_path, 'Twinning wisp', img_name)).convert('RGB')

        nt = 0
        t_poly = t.getElementsByTagName("polygon") + t.getElementsByTagName("polyline")
        for t_ply in t_poly:
            nt = nt + 1
            t_ply_label = t_ply.getAttribute("label").split('_Reflection')[0]

            # assert t_ply_label in class_list.keys()
            if not t_ply_label in class_list.keys():
                continue

            t_img = Image.new('L',(t_width,t_height))
            draw = ImageDraw.Draw(t_img)

            t_ply_points = t_ply.getAttribute("points")
            t_ply_points = t_ply_points.split(";")
            t_ply_points = np.asarray([[int(float(b)) for b in a.split(",")] for a in t_ply_points])
            t_x = t_ply_points[:,0]
            t_y = t_ply_points[:,1]
            t_xy = [(x,y) for x,y in zip(t_x,t_y)]

            draw.polygon(t_xy, fill='white')
            draw.line(t_xy, fill='white', width=2)

            t_img_np = np.array(t_img)
            lbl = label(t_img_np)
            props = regionprops(lbl)
            if len(props)>1:
                raise Exception(f'{src_path} {t_name} {nt} {t_ply_label} got {len(props)} >1 regions')
            if props[0].area<50 and not t_ply_label.lower() == 'pinpoint':
                raise Exception(f'{src_path} {t_name} {nt} {t_ply_label} should be pinpoint with area {props[0].area}')

            t_class_id = class_list[t_ply_label]

            prop = props[0]
            box = (prop.bbox[1]-clear_dist, prop.bbox[0]-clear_dist, prop.bbox[3]+clear_dist, prop.bbox[2]+clear_dist)
            # print(t_name[:11]+'_'+str(nt),t_class_id, t_ply_label)
            label_file.write('{} {} {} {}\n'.format(t_name[:11]+'_'+str(nt),t_class_id,t_ply_label,box))

            img_tmp = img.crop(box)
            if resize:
                img_tmp = img_tmp.resize(tar_size,Image.NEAREST)
            os.makedirs(os.path.join(save_path,t_ply_label),exist_ok=True)
            img_tmp.save(os.path.join(save_path,t_ply_label,t_name[:11]+'_'+str(nt)+'.png'))

        label_file.close()


In [3]:
""" 3 Generate Plain Dataset """
base_path = '/media/hdd/diamond/data/hdr_data_v2'
data_plain_path = '/media/hdd/lingjia/hdd_diamond/cls/data/plain_v2'
txt_save_path = os.path.join(data_plain_path,'txt_file')
os.makedirs(data_plain_path,exist_ok=True)
os.makedirs(txt_save_path,exist_ok=True)
# txt save info
# diamondID_regionIDX classIDX className Coordinates of four corners

# save patches with inclusion at center
for root,dirs,files in os.walk(base_path):
    status = checkXML(files)
    if status:
        print(root)
        CutAndLabel(status,root,data_plain_path,txt_save_path,resize=False,tar_size=(128,128),clear_dist=10)
        # CutAndLabel_true_inclusion(status,root,save_path,txt_save_path,resize=False,tar_size=(64,64),clear_dist=10)


/media/hdd/diamond/data/hdr_data_v2/20210129 65 HDR images and videos (1920 - 1984)/20210114 CiytU VVS2 to SI2 (A)(id211-209)
/media/hdd/diamond/data/hdr_data_v2/20210129 65 HDR images and videos (1920 - 1984)/20210119 CityU VVS2, VS2 to I1 (C)(id216-214)
/media/hdd/diamond/data/hdr_data_v2/20210129 65 HDR images and videos (1920 - 1984)/20210111 CityU VVS2 (D)(id208-206)
/media/hdd/diamond/data/hdr_data_v2/20210129 65 HDR images and videos (1920 - 1984)/20210118 CityU VVS2 to VS2 (B)(id213-211)
/media/hdd/diamond/data/hdr_data_v2/20201215 64 HDR images and videos (1715 - 1778)/20201202 CityU VVS2 to VS2 (B)(id185-184)
/media/hdd/diamond/data/hdr_data_v2/20201215 64 HDR images and videos (1715 - 1778)/20201130 CityU VS1 SI1 to SI2 (D)(id182-181)
/media/hdd/diamond/data/hdr_data_v2/20201215 64 HDR images and videos (1715 - 1778)/20201201 CityU VVS1 to SI1(A)(id184-183)
/media/hdd/diamond/data/hdr_data_v2/20201215 64 HDR images and videos (1715 - 1778)/20201208 CityU SI1 to SI2 (D)(id188

In [5]:
""" 4 Split to Train Val Test Phases """
import random
from math import floor
from shutil import copy2

# data_plain_path = '/media/hdd/lingjia/hdd_diamond/cls/20221003_resnet/plain'
# data_plain_path

classes = [n for n in os.listdir(data_plain_path) if 'txt' not in n]
classes.sort()

val_per = 0.1
test_per = 0.1
save_base_path = '/media/hdd/lingjia/hdd_diamond/cls/data/cls5'
# for exp_idx in range(1,6):
exp_idx = 1
save_dir = os.path.join(save_base_path, f'exp{exp_idx}')
for cls_idx, cls_name in enumerate(classes):
    fileList = os.listdir(os.path.join(data_plain_path,cls_name))
    fileList.sort()
    random.seed(exp_idx)
    # Split index to train & test set
    random.shuffle(fileList)
    x_test = fileList[:floor(test_per*len(fileList))]
    x_val = fileList[floor(test_per*len(fileList)):floor(test_per*len(fileList))*2]
    x_train = fileList[floor(test_per*len(fileList))*2:]
    print(f'{cls_name} all:{len(fileList)} train:{len(x_train)} val:{len(x_val)} test:{len(x_test)}')

    # Copy to save directory
    train_save_dir = os.path.join(save_dir,'train',cls_name)
    val_save_dir = os.path.join(save_dir,'val',cls_name)
    test_save_dir = os.path.join(save_dir,'test',cls_name)
    os.makedirs(train_save_dir, exist_ok=True)
    os.makedirs(val_save_dir, exist_ok=True)
    os.makedirs(test_save_dir, exist_ok=True)

    for img in x_train:
        copy2(os.path.join(data_plain_path,cls_name,img),os.path.join(train_save_dir,img))
    for img in x_val:
        copy2(os.path.join(data_plain_path,cls_name,img),os.path.join(val_save_dir,img))
    for img in x_test:
        copy2(os.path.join(data_plain_path,cls_name,img),os.path.join(test_save_dir,img))

# filelist = os.listdir('/media/hdd/lingjia/hdd_diamond/cls/20221003_resnet/diamond_set/exp1/test/Cloud')
# print(len(filelist))

Cloud all:13410 train:10728 val:1341 test:1341
Crystal all:18224 train:14580 val:1822 test:1822
Feather all:1385 train:1109 val:138 test:138
Needle all:542 train:434 val:54 test:54
Twinning_wisp all:1845 train:1477 val:184 test:184


In [3]:
""" 5 Calculate MEAN & STD """
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms

def get_mean_and_std(dataset):
    '''Compute the mean and std value of dataset.'''
    dataloader = trainloader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=True, num_workers=2)

    mean = torch.zeros(3)
    std = torch.zeros(3)
    print('==> Computing mean and std..')
    for inputs, targets in dataloader:
        for i in range(3):
            mean[i] += inputs[:,i,:,:].mean()
            std[i] += inputs[:,i,:,:].std()
    mean.div_(len(dataset))
    std.div_(len(dataset))
    return mean, std

image_size = 224
# save_path = '/media/hdd/lingjia/hdd_diamond/cls/data/cls5/exp1/train'
# save_path = '/media/hdd/lingjia/hdd_diamond/cls/data/diamond/exp11/train'
save_path = '/media/hdd/Bella/diamond_preprocessed/upscaled_diamond'
# dataset = datasets.ImageFolder(save_path, transforms.Compose([transforms.RandomResizedCrop(224),transforms.ToTensor(),]))
dataset = datasets.ImageFolder(save_path, transforms.Compose([transforms.ToTensor(),
                                                            #   transforms.Resize((image_size,image_size),interpolation=transforms.InterpolationMode.BICUBIC),
                                                              ]))
mean, std = get_mean_and_std(dataset)
print(f'mean {mean},std {std}')

# ToTensor
# tensor([0.3265, 0.3370, 0.3371]) tensor([0.1614, 0.1663, 0.1668])
# RandomResizedCrop(224)
# tensor([0.3377, 0.3490, 0.3487]) tensor([0.1371, 0.1415, 0.1426])

# 2022-10-18 version cls5/exp1 used in weightedsample
# ToTensor only
# mean tensor([0.3257, 0.3362, 0.3363]),std tensor([0.1612, 0.1660, 0.1665])
# RandomResizedCrop(224)
# tensor([0.3364, 0.3477, 0.3473]) tensor([0.1371, 0.1415, 0.1424])

# /media/hdd/lingjia/hdd_diamond/cls/data/diamond/exp11
# ToTensor only
# mean tensor([0.3268, 0.3374, 0.3375]),std tensor([0.1620, 0.1668, 0.1674])
# Resize transforms.InterpolationMode.BICUBIC
# mean tensor([0.3268, 0.3374, 0.3375]),std tensor([0.1622, 0.1671, 0.1676])

# /media/hdd/Bella/diamond_preprocessed/upscaled_diamond
# Resize transforms.InterpolationMode.BICUBIC
# mean tensor([0.3287, 0.3395, 0.3398]),std tensor([0.1749, 0.1769, 0.1787])


==> Computing mean and std..
mean tensor([0.3287, 0.3395, 0.3398]),std tensor([0.1749, 0.1768, 0.1787])
